In [1]:
import pandas as pd
import numpy as np
import datetime

import matplotlib.pyplot as plt
import sklearn


# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

import tensorflow as tf

# Keras specific
import keras
from keras.models import Sequential
from keras.layers import Dense

In [2]:
#!pip install tensorflow

#TODO - add day of year


In [3]:
#load data from data files and place into panda dataframes
custOrders = pd.read_csv('data/customer_orders.csv', low_memory=False)
temp2 = pd.read_csv('data/item_ingredients.csv', low_memory=False)
itemID=pd.read_csv('../SQL/item.csv')

In [4]:
#turn off panda max column display limit
print(pd.options.display.max_columns)
pd.options.display.max_columns
mns = None


20


In [5]:
#convert the boolean values into integers in the ingredients dataframe
#also print column names of dataframe

#manually arranges menu items onto a numberline
itemID=itemID.reindex([5,6,2,1,4,0,3,19,13,14,15,16,17,18,   7,8,9,10,11,12,20])

itemID['item_score'] = range(1, len(itemID) + 1)
itemID = itemID.drop("item_description", axis=1, errors='ignore')
itemID = itemID.drop("item_image", axis=1, errors='ignore')


#itemID = itemID.reset_index()
itemID_Dict={}
for i, row in itemID.iterrows():
    itemID_Dict[row.loc["item_id"]]=(row.loc["item_score"], row.loc["item_name"])

print(itemID_Dict)
itemID.head(100)


{640405112: (1, 'Double Cheeseburger'), 640405172: (2, 'Double Bacon Cheeseburger'), 640405025: (3, 'Bacon Cheeseburger'), 640404963: (4, 'Cheeseburger'), 640405085: (5, 'Double Hamburger'), 640404923: (6, 'Hamburger'), 640405058: (7, 'Veggie Burger'), 640405347: (8, 'Onion Rings'), 640405296: (9, 'Small Fries'), 640405307: (10, 'Regular Fries'), 640405315: (11, 'Large Fries'), 640405323: (12, 'Small Curly Fries'), 640405331: (13, 'Regular Curly Fries'), 640405339: (14, 'Large Curly Fries'), 640405355: (15, 'Small Drink'), 640405371: (16, 'Regular Drink'), 640405380: (17, 'Large Drink'), 640405389: (18, 'Small Shake'), 640405395: (19, 'Regular Shake'), 640405399: (20, 'Large Shake'), 640405348: (21, 'Coffee')}


,item_id,item_name,price,taste_profile,item_type,item_score
5,640405112,Double Cheeseburger,3.29,savory,burger,1
6,640405172,Double Bacon Cheeseburger,3.79,savory,burger,2
2,640405025,Bacon Cheeseburger,2.79,savory,burger,3
1,640404963,Cheeseburger,2.29,savory,burger,4
4,640405085,Double Hamburger,2.99,savory,burger,5
0,640404923,Hamburger,3.07,savory,burger,6
3,640405058,Veggie Burger,2.49,healthy,burger,7
19,640405347,Onion Rings,2.19,sweet,side,8
13,640405296,Small Fries,1.49,savory,side,9
14,640405307,Regular Fries,1.79,savory,side,10


In [6]:
#print the binary version of the item-ingredient dataframe
temp2.head()
temp2.shape

(22, 19)

In [7]:
#change type to datetime
custOrders["order_timestamp"]=pd.to_datetime(custOrders["order_timestamp"])
custOrders.head()

,customer_id,order_id,weather,order_timestamp,item_id,item_name,taste_profile,item_type
0,52,1752,snowy,2015-01-02 20:38:35,640405323,Small Curly Fries,savory,side
1,52,1752,snowy,2015-01-02 20:38:35,640405331,Regular Curly Fries,savory,side
2,194,244,snowy,2015-01-04 16:51:57,640405315,Large Fries,savory,side
3,1,714,cloudy,2015-01-05 20:58:57,640405307,Regular Fries,savory,side
4,1,714,cloudy,2015-01-05 20:58:57,640405355,Small Drink,chilling,beverage


In [8]:
custOrders.head()

,customer_id,order_id,weather,order_timestamp,item_id,item_name,taste_profile,item_type
0,52,1752,snowy,2015-01-02 20:38:35,640405323,Small Curly Fries,savory,side
1,52,1752,snowy,2015-01-02 20:38:35,640405331,Regular Curly Fries,savory,side
2,194,244,snowy,2015-01-04 16:51:57,640405315,Large Fries,savory,side
3,1,714,cloudy,2015-01-05 20:58:57,640405307,Regular Fries,savory,side
4,1,714,cloudy,2015-01-05 20:58:57,640405355,Small Drink,chilling,beverage


In [9]:
#? unsure ----> item_name, item_description
#Think need more item related features, 

#function converts a cyclical data point(like time) into a 2 point representation on a circle 
def cycleEmbed(maximum, reading):
    maxi=maximum

    sin = np.sin(2*np.pi*reading/maxi)
    cos = np.cos(2*np.pi*reading/maxi)
    return (sin, cos)



In [10]:
#function takes a column name, dataframe, and then converts the column in the dataframe into a integers, 
#where each integer represents a unique value from that column
def enumerateCol(colName, dataFrame):
    count=0
    diction={}

    for i, row in dataFrame.iterrows():
        currentVal=row.loc[colName]
        if currentVal in diction:
            dataFrame.at[i, colName] = diction[currentVal]
        else:
            diction[currentVal]=count
            count+=1
            dataFrame.at[i, colName] = diction[currentVal]
            
#function enumerates column based on the itemID_Dict to 
def enumerateItemID(colName, dataFrame):
    count=0
    diction=itemID_Dict
    print(diction)
    for i, row in dataFrame.iterrows():
        currentVal=row.loc[colName]
        dataFrame.at[i, colName] = diction[currentVal][0]



In [11]:
# Get a Dictionary containing the pairs of column names & data type objects.

print('Data type of each column of Dataframe :')
print(dict(custOrders.dtypes))


Data type of each column of Dataframe :
{'customer_id': dtype('int64'), 'order_id': dtype('int64'), 'weather': dtype('O'), 'order_timestamp': dtype('<M8[ns]'), 'item_id': dtype('int64'), 'item_name': dtype('O'), 'taste_profile': dtype('O'), 'item_type': dtype('O')}


In [12]:
#split timestamp in "order_timestamp" into a number of individual columns for each part of the timestamp
custOrders["order_timestamp"]=pd.to_datetime(custOrders["order_timestamp"])
custOrders["order_date"]=custOrders["order_timestamp"].dt.date
custOrders["order_year"]=custOrders["order_timestamp"].dt.year
custOrders["order_month"]=custOrders["order_timestamp"].dt.month
custOrders["order_day"]=custOrders["order_timestamp"].dt.day
custOrders["order_hour"]=custOrders["order_timestamp"].dt.hour
custOrders["order_minute"]=custOrders["order_timestamp"].dt.minute
custOrders["order_second"]=custOrders["order_timestamp"].dt.second

#create a column for the weekday(mon,tues, weds, ...etc)
#must convert to weekday below
custOrders["order_weekday"]=custOrders["order_day"]

for i, row in custOrders.iterrows():
    custOrders.at[i, "order_weekday"] = datetime.datetime(row["order_year"], row["order_month"],row["order_day"],0,0,0).weekday()


In [13]:
#drop columns which are optional(not all customers will use it), clearly not useful, or redundant

custOrders = custOrders.drop("item_image", axis=1, errors='ignore')
custOrders = custOrders.drop("order_timestamp", axis=1, errors='ignore') #information moved into other columns added above
custOrders = custOrders.drop("customization_id", axis=1, errors='ignore') #customization is no in every order
custOrders = custOrders.drop("customization", axis=1, errors='ignore')
custOrders = custOrders.drop("opt_in", axis=1, errors='ignore')
custOrders = custOrders.drop("item_description", axis=1, errors='ignore')

custOrders = custOrders.drop("birthday", axis=1, errors='ignore') #check if part of opt in
custOrders = custOrders.drop("gender", axis=1, errors='ignore') #check if part of opt in
#custOrders = custOrders.drop("item_name", axis=1) #reduntance since we have item id? we need 

custOrders = custOrders.drop("order_date", axis=1, errors='ignore') #check if part of opt in


custOrders.head(10)


,customer_id,order_id,weather,item_id,item_name,taste_profile,item_type,order_year,order_month,order_day,order_hour,order_minute,order_second,order_weekday
0,52,1752,snowy,640405323,Small Curly Fries,savory,side,2015,1,2,20,38,35,4
1,52,1752,snowy,640405331,Regular Curly Fries,savory,side,2015,1,2,20,38,35,4
2,194,244,snowy,640405315,Large Fries,savory,side,2015,1,4,16,51,57,6
3,1,714,cloudy,640405307,Regular Fries,savory,side,2015,1,5,20,58,57,0
4,1,714,cloudy,640405355,Small Drink,chilling,beverage,2015,1,5,20,58,57,0
5,1,714,cloudy,640404963,Cheeseburger,savory,burger,2015,1,5,20,58,57,0
6,1,714,cloudy,640405347,Onion Rings,sweet,side,2015,1,5,20,58,57,0
7,47,1546,sunny,640405371,Regular Drink,chilling,beverage,2015,1,8,19,49,40,3
8,47,1546,sunny,640405395,Regular Shake,sweet,beverage,2015,1,8,19,49,40,3
9,47,1546,sunny,640405085,Double Hamburger,savory,burger,2015,1,8,19,49,40,3


In [14]:
#custOrders will represet the data before it has been enumerated and is only numbers
custOrders2=custOrders.copy()
custOrders2.head()

,customer_id,order_id,weather,item_id,item_name,taste_profile,item_type,order_year,order_month,order_day,order_hour,order_minute,order_second,order_weekday
0,52,1752,snowy,640405323,Small Curly Fries,savory,side,2015,1,2,20,38,35,4
1,52,1752,snowy,640405331,Regular Curly Fries,savory,side,2015,1,2,20,38,35,4
2,194,244,snowy,640405315,Large Fries,savory,side,2015,1,4,16,51,57,6
3,1,714,cloudy,640405307,Regular Fries,savory,side,2015,1,5,20,58,57,0
4,1,714,cloudy,640405355,Small Drink,chilling,beverage,2015,1,5,20,58,57,0


In [15]:
#embed time values to cycles on a circle
def timeEmbed(df, colName, maxi):
    name1=colName+"1"
    name2=colName+"2"
    print(name1)
    print(name2)
    df[name1]=None
    df[name2]=None
    for i, row in df.iterrows():
        temp=cycleEmbed(maxi, row[colName])
        df.at[i, name1]=temp[0]
        df.at[i, name2]=temp[1]

#convert every time based column into a point on a circle with timeEmbed()    
timeEmbed(custOrders2, "order_month", 12)
timeEmbed(custOrders2, "order_day", 31)
timeEmbed(custOrders2, "order_hour", 24)
timeEmbed(custOrders2, "order_minute", 60)
timeEmbed(custOrders2, "order_second", 60)
timeEmbed(custOrders2, "order_weekday", 7)



order_month1
order_month2
order_day1
order_day2
order_hour1
order_hour2
order_minute1
order_minute2
order_second1
order_second2
order_weekday1
order_weekday2


In [16]:
#print all number version of the data

custOrders2.head(10)

,customer_id,order_id,weather,item_id,item_name,taste_profile,item_type,order_year,order_month,order_day,...,order_day1,order_day2,order_hour1,order_hour2,order_minute1,order_minute2,order_second1,order_second2,order_weekday1,order_weekday2
0,52,1752,snowy,640405323,Small Curly Fries,savory,side,2015,1,2,...,0.394356,0.918958,-0.866025,0.5,-0.743145,-0.669131,-0.5,-0.866025,-0.433884,-0.900969
1,52,1752,snowy,640405331,Regular Curly Fries,savory,side,2015,1,2,...,0.394356,0.918958,-0.866025,0.5,-0.743145,-0.669131,-0.5,-0.866025,-0.433884,-0.900969
2,194,244,snowy,640405315,Large Fries,savory,side,2015,1,4,...,0.724793,0.688967,-0.866025,-0.5,-0.809017,0.587785,-0.309017,0.951057,-0.781831,0.62349
3,1,714,cloudy,640405307,Regular Fries,savory,side,2015,1,5,...,0.848644,0.528964,-0.866025,0.5,-0.207912,0.978148,-0.309017,0.951057,0.0,1.0
4,1,714,cloudy,640405355,Small Drink,chilling,beverage,2015,1,5,...,0.848644,0.528964,-0.866025,0.5,-0.207912,0.978148,-0.309017,0.951057,0.0,1.0
5,1,714,cloudy,640404963,Cheeseburger,savory,burger,2015,1,5,...,0.848644,0.528964,-0.866025,0.5,-0.207912,0.978148,-0.309017,0.951057,0.0,1.0
6,1,714,cloudy,640405347,Onion Rings,sweet,side,2015,1,5,...,0.848644,0.528964,-0.866025,0.5,-0.207912,0.978148,-0.309017,0.951057,0.0,1.0
7,47,1546,sunny,640405371,Regular Drink,chilling,beverage,2015,1,8,...,0.998717,-0.050649,-0.965926,0.258819,-0.913545,0.406737,-0.866025,-0.5,0.433884,-0.900969
8,47,1546,sunny,640405395,Regular Shake,sweet,beverage,2015,1,8,...,0.998717,-0.050649,-0.965926,0.258819,-0.913545,0.406737,-0.866025,-0.5,0.433884,-0.900969
9,47,1546,sunny,640405085,Double Hamburger,savory,burger,2015,1,8,...,0.998717,-0.050649,-0.965926,0.258819,-0.913545,0.406737,-0.866025,-0.5,0.433884,-0.900969


In [17]:
#begin splitting custOrders2 into training/validation sets, including moving supervision target(itemID) out

masterDS=custOrders2.copy()
masterDS = masterDS.drop("order_month", axis=1, errors='ignore')
masterDS = masterDS.drop("order_day", axis=1, errors='ignore')
masterDS = masterDS.drop("order_hour", axis=1, errors='ignore')
masterDS = masterDS.drop("order_year", axis=1, errors='ignore')
masterDS = masterDS.drop("order_minute", axis=1, errors='ignore')
masterDS = masterDS.drop("order_second", axis=1, errors='ignore')
masterDS = masterDS.drop("order_weekday", axis=1, errors='ignore')


masterDS = masterDS.drop("item_image", axis=1, errors='ignore')
masterDS = masterDS.drop("birthday", axis=1, errors='ignore')


#masterDS = masterDS.drop("customer_id", axis=1, errors='ignore')
#masterDS = masterDS.drop("order_id", axis=1, errors='ignore')



In [18]:
masterDS.head()

,customer_id,order_id,weather,item_id,item_name,taste_profile,item_type,order_month1,order_month2,order_day1,order_day2,order_hour1,order_hour2,order_minute1,order_minute2,order_second1,order_second2,order_weekday1,order_weekday2
0,52,1752,snowy,640405323,Small Curly Fries,savory,side,0.5,0.866025,0.394356,0.918958,-0.866025,0.5,-0.743145,-0.669131,-0.5,-0.866025,-0.433884,-0.900969
1,52,1752,snowy,640405331,Regular Curly Fries,savory,side,0.5,0.866025,0.394356,0.918958,-0.866025,0.5,-0.743145,-0.669131,-0.5,-0.866025,-0.433884,-0.900969
2,194,244,snowy,640405315,Large Fries,savory,side,0.5,0.866025,0.724793,0.688967,-0.866025,-0.5,-0.809017,0.587785,-0.309017,0.951057,-0.781831,0.62349
3,1,714,cloudy,640405307,Regular Fries,savory,side,0.5,0.866025,0.848644,0.528964,-0.866025,0.5,-0.207912,0.978148,-0.309017,0.951057,0.0,1.0
4,1,714,cloudy,640405355,Small Drink,chilling,beverage,0.5,0.866025,0.848644,0.528964,-0.866025,0.5,-0.207912,0.978148,-0.309017,0.951057,0.0,1.0


In [19]:

#Enumerate columns that are categorical
enumerateCol("weather", masterDS)
enumerateItemID("item_id", masterDS)
enumerateCol("taste_profile", masterDS)
enumerateCol("item_type", masterDS)

masterDS.head()

masterDS = masterDS.drop("item_name", axis=1, errors='ignore')


{640405112: (1, 'Double Cheeseburger'), 640405172: (2, 'Double Bacon Cheeseburger'), 640405025: (3, 'Bacon Cheeseburger'), 640404963: (4, 'Cheeseburger'), 640405085: (5, 'Double Hamburger'), 640404923: (6, 'Hamburger'), 640405058: (7, 'Veggie Burger'), 640405347: (8, 'Onion Rings'), 640405296: (9, 'Small Fries'), 640405307: (10, 'Regular Fries'), 640405315: (11, 'Large Fries'), 640405323: (12, 'Small Curly Fries'), 640405331: (13, 'Regular Curly Fries'), 640405339: (14, 'Large Curly Fries'), 640405355: (15, 'Small Drink'), 640405371: (16, 'Regular Drink'), 640405380: (17, 'Large Drink'), 640405389: (18, 'Small Shake'), 640405395: (19, 'Regular Shake'), 640405399: (20, 'Large Shake'), 640405348: (21, 'Coffee')}


In [20]:
#? need to normalize?
# DO I NEED TO DO the FLOAT 64 CAST? 
#cast all columns to float64

for col in masterDS.columns:
    masterDS[col] = masterDS[col].astype(np.float64)
masterDS.head(10)


,customer_id,order_id,weather,item_id,taste_profile,item_type,order_month1,order_month2,order_day1,order_day2,order_hour1,order_hour2,order_minute1,order_minute2,order_second1,order_second2,order_weekday1,order_weekday2
0,52.0,1752.0,0.0,12.0,0.0,0.0,0.5,0.866025,0.394356,0.918958,-0.866025,0.500000,-0.743145,-0.669131,-0.500000,-0.866025,-0.433884,-0.900969
1,52.0,1752.0,0.0,13.0,0.0,0.0,0.5,0.866025,0.394356,0.918958,-0.866025,0.500000,-0.743145,-0.669131,-0.500000,-0.866025,-0.433884,-0.900969
2,194.0,244.0,0.0,11.0,0.0,0.0,0.5,0.866025,0.724793,0.688967,-0.866025,-0.500000,-0.809017,0.587785,-0.309017,0.951057,-0.781831,0.623490
3,1.0,714.0,1.0,10.0,0.0,0.0,0.5,0.866025,0.848644,0.528964,-0.866025,0.500000,-0.207912,0.978148,-0.309017,0.951057,0.000000,1.000000
4,1.0,714.0,1.0,15.0,1.0,1.0,0.5,0.866025,0.848644,0.528964,-0.866025,0.500000,-0.207912,0.978148,-0.309017,0.951057,0.000000,1.000000
5,1.0,714.0,1.0,4.0,0.0,2.0,0.5,0.866025,0.848644,0.528964,-0.866025,0.500000,-0.207912,0.978148,-0.309017,0.951057,0.000000,1.000000
6,1.0,714.0,1.0,8.0,2.0,0.0,0.5,0.866025,0.848644,0.528964,-0.866025,0.500000,-0.207912,0.978148,-0.309017,0.951057,0.000000,1.000000
7,47.0,1546.0,2.0,16.0,1.0,1.0,0.5,0.866025,0.998717,-0.050649,-0.965926,0.258819,-0.913545,0.406737,-0.866025,-0.500000,0.433884,-0.900969
8,47.0,1546.0,2.0,19.0,2.0,1.0,0.5,0.866025,0.998717,-0.050649,-0.965926,0.258819,-0.913545,0.406737,-0.866025,-0.500000,0.433884,-0.900969
9,47.0,1546.0,2.0,5.0,0.0,2.0,0.5,0.866025,0.998717,-0.050649,-0.965926,0.258819,-0.913545,0.406737,-0.866025,-0.500000,0.433884,-0.900969


In [21]:
target_column = ['item_id'] 
predictors = list(set(list(masterDS.columns))-set(target_column))
print(predictors)

#next line regularizes the all columns outside of target_column
# I definitly dont want it for the time data points, but do other columns need it?????

#masterDS[predictors] = masterDS[predictors]/masterDS[predictors].max()

['order_hour1', 'customer_id', 'weather', 'order_month2', 'order_minute2', 'order_day1', 'order_day2', 'taste_profile', 'order_hour2', 'item_type', 'order_second1', 'order_weekday1', 'order_minute1', 'order_month1', 'order_weekday2', 'order_id', 'order_second2']


In [22]:
X = masterDS[predictors].values
y = masterDS[target_column].values

In [23]:
#split into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=40)
print(X_train.shape); print(X_test.shape)

(3847, 17)
(962, 17)


In [24]:
# Define model
model = Sequential()
model.add(Dense(500, input_dim=X_train.shape[1], activation= "relu"))
model.add(Dense(100, activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1))
#model.summary() #Print model Summary


In [25]:
#tf.keras.backend.set_floatx('float64')

model.compile(loss= "mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

model.fit(X_train, y_train, epochs=300)


Epoch 1/300
121/121 [==============================] - 1s 3ms/step - loss: 103.7838 - mean_squared_error: 103.7838
Epoch 2/300
121/121 [==============================] - 1s 4ms/step - loss: 51.6872 - mean_squared_error: 51.6872
Epoch 3/300
121/121 [==============================] - 1s 4ms/step - loss: 47.1834 - mean_squared_error: 47.1834
Epoch 4/300
121/121 [==============================] - 1s 5ms/step - loss: 44.5830 - mean_squared_error: 44.5830
Epoch 5/300
121/121 [==============================] - 1s 5ms/step - loss: 46.9919 - mean_squared_error: 46.9919
Epoch 6/300
121/121 [==============================] - 1s 4ms/step - loss: 37.8389 - mean_squared_error: 37.8389
Epoch 7/300
121/121 [==============================] - 1s 5ms/step - loss: 36.2131 - mean_squared_error: 36.2131
Epoch 8/300
121/121 [==============================] - 1s 4ms/step - loss: 32.6379 - mean_squared_error: 32.6379
Epoch 9/300
121/121 [==============================] - 1s 4ms/step - loss: 32.1476 - mean_squa

121/121 [==============================] - 0s 2ms/step - loss: 13.3740 - mean_squared_error: 13.3740
Epoch 73/300
121/121 [==============================] - 0s 2ms/step - loss: 11.4033 - mean_squared_error: 11.4033
Epoch 74/300
121/121 [==============================] - 0s 2ms/step - loss: 11.5243 - mean_squared_error: 11.5243
Epoch 75/300
121/121 [==============================] - 0s 2ms/step - loss: 12.3965 - mean_squared_error: 12.3965
Epoch 76/300
121/121 [==============================] - 0s 2ms/step - loss: 12.7435 - mean_squared_error: 12.7435
Epoch 77/300
121/121 [==============================] - 0s 2ms/step - loss: 12.5102 - mean_squared_error: 12.5102
Epoch 78/300
121/121 [==============================] - 0s 2ms/step - loss: 10.8862 - mean_squared_error: 10.8862
Epoch 79/300
121/121 [==============================] - 0s 2ms/step - loss: 10.6576 - mean_squared_error: 10.6576
Epoch 80/300
121/121 [==============================] - 0s 2ms/step - loss: 11.8765 - mean_squared_er

121/121 [==============================] - 0s 3ms/step - loss: 2.5113 - mean_squared_error: 2.5113
Epoch 145/300
121/121 [==============================] - 0s 2ms/step - loss: 5.3940 - mean_squared_error: 5.3940
Epoch 146/300
121/121 [==============================] - 0s 4ms/step - loss: 4.4989 - mean_squared_error: 4.4989
Epoch 147/300
121/121 [==============================] - 0s 2ms/step - loss: 2.8290 - mean_squared_error: 2.8290
Epoch 148/300
121/121 [==============================] - 0s 2ms/step - loss: 2.2153 - mean_squared_error: 2.2153
Epoch 149/300
121/121 [==============================] - 0s 2ms/step - loss: 2.0590 - mean_squared_error: 2.0590
Epoch 150/300
121/121 [==============================] - 0s 2ms/step - loss: 2.1164 - mean_squared_error: 2.1164
Epoch 151/300
121/121 [==============================] - 0s 2ms/step - loss: 1.9806 - mean_squared_error: 1.9806
Epoch 152/300
121/121 [==============================] - 0s 2ms/step - loss: 1.9858 - mean_squared_error: 1.98

121/121 [==============================] - 0s 2ms/step - loss: 3.9265 - mean_squared_error: 3.9265
Epoch 217/300
121/121 [==============================] - 0s 2ms/step - loss: 1.8748 - mean_squared_error: 1.8748
Epoch 218/300
121/121 [==============================] - 0s 4ms/step - loss: 1.8673 - mean_squared_error: 1.8673
Epoch 219/300
121/121 [==============================] - 0s 4ms/step - loss: 1.8359 - mean_squared_error: 1.8359
Epoch 220/300
121/121 [==============================] - 0s 4ms/step - loss: 1.8354 - mean_squared_error: 1.8354
Epoch 221/300
121/121 [==============================] - 0s 4ms/step - loss: 1.8165 - mean_squared_error: 1.8165
Epoch 222/300
121/121 [==============================] - 0s 3ms/step - loss: 1.8149 - mean_squared_error: 1.8149
Epoch 223/300
121/121 [==============================] - 0s 2ms/step - loss: 1.8357 - mean_squared_error: 1.8357
Epoch 224/300
121/121 [==============================] - 0s 2ms/step - loss: 1.8318 - mean_squared_error: 1.83

121/121 [==============================] - 0s 2ms/step - loss: 1.7451 - mean_squared_error: 1.7451
Epoch 289/300
121/121 [==============================] - 0s 2ms/step - loss: 1.7421 - mean_squared_error: 1.7421
Epoch 290/300
121/121 [==============================] - 0s 2ms/step - loss: 2.0983 - mean_squared_error: 2.0983
Epoch 291/300
121/121 [==============================] - 0s 2ms/step - loss: 1.7538 - mean_squared_error: 1.7538
Epoch 292/300
121/121 [==============================] - 0s 2ms/step - loss: 1.7647 - mean_squared_error: 1.7647
Epoch 293/300
121/121 [==============================] - 0s 3ms/step - loss: 1.7388 - mean_squared_error: 1.7388
Epoch 294/300
121/121 [==============================] - 0s 3ms/step - loss: 1.7943 - mean_squared_error: 1.7943
Epoch 295/300
121/121 [==============================] - 0s 3ms/step - loss: 1.8506 - mean_squared_error: 1.8506
Epoch 296/300
121/121 [==============================] - 0s 3ms/step - loss: 1.7468 - mean_squared_error: 1.74

In [26]:
# Get a Dictionary containing the pairs of column names & data type objects.
print('Data type of each column of Dataframe :')
print(dict(masterDS.dtypes))

Data type of each column of Dataframe :
{'customer_id': dtype('float64'), 'order_id': dtype('float64'), 'weather': dtype('float64'), 'item_id': dtype('float64'), 'taste_profile': dtype('float64'), 'item_type': dtype('float64'), 'order_month1': dtype('float64'), 'order_month2': dtype('float64'), 'order_day1': dtype('float64'), 'order_day2': dtype('float64'), 'order_hour1': dtype('float64'), 'order_hour2': dtype('float64'), 'order_minute1': dtype('float64'), 'order_minute2': dtype('float64'), 'order_second1': dtype('float64'), 'order_second2': dtype('float64'), 'order_weekday1': dtype('float64'), 'order_weekday2': dtype('float64')}


In [27]:
pred_train= model.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_train)))

#model.predict([40.0,139.0, 0.0, 4.99, 0.00, ])

print(pred_train)
pd.DataFrame(pred_train).head


1.484482446460314
[[16.201416  ]
 [ 3.5821662 ]
 [ 7.6188345 ]
 [21.42186   ]
 [ 3.520872  ]
 [11.699894  ]
 [19.151644  ]
 [16.014765  ]
 [ 3.4788225 ]
 [11.719923  ]
 [ 3.5796793 ]
 [15.987271  ]
 [ 3.638366  ]
 [16.026148  ]
 [16.064447  ]
 [15.972255  ]
 [19.187706  ]
 [11.794941  ]
 [ 2.9596214 ]
 [16.006668  ]
 [21.481348  ]
 [ 8.086456  ]
 [ 7.7338276 ]
 [20.6325    ]
 [11.797814  ]
 [ 3.2493744 ]
 [11.77181   ]
 [11.744959  ]
 [19.523365  ]
 [ 3.749228  ]
 [19.288176  ]
 [16.116076  ]
 [ 2.2862582 ]
 [16.113226  ]
 [ 3.4860344 ]
 [21.541456  ]
 [11.687425  ]
 [11.776723  ]
 [19.110794  ]
 [15.996136  ]
 [16.031858  ]
 [ 3.4967558 ]
 [11.953357  ]
 [ 3.5763893 ]
 [ 3.5542302 ]
 [ 3.443122  ]
 [ 7.7795143 ]
 [ 7.3439093 ]
 [ 3.7359104 ]
 [ 3.3768215 ]
 [ 7.4002066 ]
 [11.844688  ]
 [11.751951  ]
 [11.821039  ]
 [16.13252   ]
 [ 3.4825678 ]
 [16.114838  ]
 [ 3.072657  ]
 [ 3.5158298 ]
 [16.066614  ]
 [21.42595   ]
 [16.187826  ]
 [ 3.436226  ]
 [16.349997  ]
 [19.067966  ]
 [ 3.40

<bound method NDFrame.head of              0
0    16.201416
1     3.582166
2     7.618834
3    21.421860
4     3.520872
..         ...
957   2.032620
958  21.520733
959   3.458019
960   1.957366
961   2.571495

[962 rows x 1 columns]>

In [28]:
pd.DataFrame(X_test).head()

# X_test, y_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,-0.965926,148.0,0.0,-8.660254e-01,-0.309017,-0.897805,-0.440394,1.0,0.258819,1.0,-0.587785,-0.781831,-0.951057,-5.000000e-01,0.623490,1331.0,0.809017
1,0.258819,17.0,3.0,-8.660254e-01,-0.587785,0.968077,-0.250653,0.0,-0.965926,2.0,-0.309017,0.781831,-0.809017,5.000000e-01,0.623490,1888.0,-0.951057
2,-0.500000,64.0,2.0,6.123234e-17,0.500000,0.651372,-0.758758,4.0,-0.866025,2.0,0.500000,0.781831,-0.866025,1.000000e+00,0.623490,26.0,0.866025
3,0.500000,189.0,2.0,8.660254e-01,-0.587785,0.299363,-0.954139,3.0,0.866025,1.0,-0.866025,-0.974928,0.809017,-5.000000e-01,-0.222521,1288.0,0.500000
4,0.500000,110.0,3.0,-1.000000e+00,1.000000,0.937752,0.347305,0.0,-0.866025,2.0,-0.809017,-0.974928,0.000000,1.224647e-16,-0.222521,832.0,0.587785


In [29]:
code_to_item={}
for pair in itemID_Dict:
    code_to_item[itemID_Dict[pair][0]]=itemID_Dict[pair][1]
    
print(code_to_item)

{1: 'Double Cheeseburger', 2: 'Double Bacon Cheeseburger', 3: 'Bacon Cheeseburger', 4: 'Cheeseburger', 5: 'Double Hamburger', 6: 'Hamburger', 7: 'Veggie Burger', 8: 'Onion Rings', 9: 'Small Fries', 10: 'Regular Fries', 11: 'Large Fries', 12: 'Small Curly Fries', 13: 'Regular Curly Fries', 14: 'Large Curly Fries', 15: 'Small Drink', 16: 'Regular Drink', 17: 'Large Drink', 18: 'Small Shake', 19: 'Regular Shake', 20: 'Large Shake', 21: 'Coffee'}


In [30]:
results=pd.DataFrame(y_test)
results["prediction"]=(pred_train[:,0].astype(float)).tolist()

results.rename(columns={0:"true"}, inplace=True)


results["true_item_code"]=None
results["prediction_item_code"]=None
results["true_item_name"]=None
results["prediction_item_name"]=None

for i, row in results.iterrows():
    print(i)
    v1= min(round(row["true"]),len(code_to_item))
    v2= min(round(row["prediction"]),len(code_to_item))
#     if(v1<0):
#         v1=1
#     if(v2<0):
#         v2=1
    
    results.at[i, "true_item_code"] = v1
    results.at[i, "prediction_item_code"] = v2 
    
    results.at[i, "true_item_name"] = code_to_item[v1]
    results.at[i, "prediction_item_name"] = code_to_item[v2]
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [31]:
results.head(25)


,true,prediction,true_item_code,prediction_item_code,true_item_name,prediction_item_name
0,16.0,16.201416,16,16,Regular Drink,Regular Drink
1,3.0,3.582166,3,4,Bacon Cheeseburger,Cheeseburger
2,7.0,7.618834,7,8,Veggie Burger,Onion Rings
3,21.0,21.421860,21,21,Coffee,Coffee
4,5.0,3.520872,5,4,Double Hamburger,Cheeseburger
5,13.0,11.699894,13,12,Regular Curly Fries,Small Curly Fries
6,19.0,19.151644,19,19,Regular Shake,Regular Shake
7,15.0,16.014765,15,16,Small Drink,Regular Drink
8,2.0,3.478822,2,3,Double Bacon Cheeseburger,Bacon Cheeseburger
9,14.0,11.719923,14,12,Large Curly Fries,Small Curly Fries


In [32]:
results.shape

(962, 6)

In [33]:
results.head(236)


,true,prediction,true_item_code,prediction_item_code,true_item_name,prediction_item_name
0,16.0,16.201416,16,16,Regular Drink,Regular Drink
1,3.0,3.582166,3,4,Bacon Cheeseburger,Cheeseburger
2,7.0,7.618834,7,8,Veggie Burger,Onion Rings
3,21.0,21.421860,21,21,Coffee,Coffee
4,5.0,3.520872,5,4,Double Hamburger,Cheeseburger
...,...,...,...,...,...,...
231,21.0,21.302778,21,21,Coffee,Coffee
232,6.0,2.941252,6,3,Hamburger,Bacon Cheeseburger
233,15.0,16.119257,15,16,Small Drink,Regular Drink
234,17.0,16.069185,17,16,Large Drink,Regular Drink
